In [ ]:
# Print the number of missing values before replacement
print("\nNumber of missing values in 'price' before replacement:", missing_price_mask.sum())

# Fill missing 'price' values based on conditions
for index in df_initial[missing_price_mask].index:
    listing_id = df_initial.at[index, 'id']  # Get the listing_id from df_initial
    calendar_date = df_initial.at[index, 'calendar_last_scraped']  # Get the date from df_initial

    # Check for the corresponding listing in the calendar dataframe
    matching_calendar_row = df_calendar[(df_calendar['listing_id'] == listing_id) & 
                                        (df_calendar['date'] == calendar_date)]
    
    # If a matching row is found, fill the missing price
    if not matching_calendar_row.empty:
        df_initial.at[index, 'price'] = matching_calendar_row['price'].values[0]  # Fill missing price

# Check for missing values in 'price' again
missing_values_after = df_initial['price'].isnull().sum()
print("\nNumber of missing values in 'price' after replacement:", missing_values_after)


In [ ]:
import dask.dataframe as dd


# Convert to Dask DataFrames for parallel processing
dask_initial = dd.from_pandas(df_initial, npartitions=4)  # Adjust npartitions as needed
dask_calendar = dd.from_pandas(df_calendar, npartitions=4)

# Function to fill missing prices
def fill_missing_prices(row):
    if pd.isnull(row['price']):
        # Check for matching listing_id and date
        matching_row = dask_calendar[
            (dask_calendar['listing_id'] == row['id']) & 
            (dask_calendar['date'] == row['calendar_last_scraped'])
        ].compute()
        
        if not matching_row.empty:
            return matching_row['price'].values[0]
    return row['price']

# Apply the function to fill missing prices
dask_initial['price'] = dask_initial.map_partitions(
    lambda df: df.apply(fill_missing_prices, axis=1),
    meta=df_initial
)

# Compute the result back to a pandas DataFrame
df_initial_filled = dask_initial.compute()

# Check for missing values in 'price' again
missing_values_after = df_initial_filled['price'].isnull().sum()
print("\nNumber of missing values in 'price' after replacement:", missing_values_after)

# Calculate the new percentage of missing values in the 'price' column
missing_percentage_after = (missing_values_after / len(df_initial_filled)) * 100
print(f"\nNew percentage of missing values in 'price' column: {missing_percentage_after:.2f}%")
